# Objective
To automatically categorize/tag TED Talk videos based on a series of features provided in a sample dataset, link below.
Once the text data has been cleaned and prepared, various Classification models will be trained on the sample dataset.

There are two datasets provided: the main summary and the transcripts

**TED Talks: Summary**
https://www.kaggle.com/rounakbanik/ted-talks?select=ted_main.csv

**TED Talks: Transcripts**
https://www.kaggle.com/rounakbanik/ted-talks?select=transcripts.csv

# Problem Statement

The data provided, in its raw format, does not make for easy classification and statistical analysis of the various TED Talks. An automated, and simplified, approach that could quickly classify (tag) a given TED talk, based on other similar feature patterns, would allow for improved viewer experiences from accurate user preference alignment.

### Data Provided - Summary:

- **Comments:** The number of first level comments made on the talk
- **Description:** A blurb of what the talk is about
- **Duration:** The duration of the talk in seconds
- **Event:** The TED/TEDx event where the talk took place
- **Film_date:** The Unix timestamp of the filming
- **Languages:** The number of languages in which the talk is available
- **Main_speaker:** The first named speaker of the talk
- **Name:** The official name of the TED Talk; includes the title and the speaker
- **Num_speaker:** The number of speakers in the talk
- **Published_date:** The Unix timestamp for the publication of the talk on TED.com
- **Ratings:** Groups of ratings assinged to each talk
- **Related_talks:** Similar talks/links
- **Speaker_occupation:** Primary speaker occupation/focus
- **Tags:** Tags assinged to the talk
- **Title:** Title of TED talk
- **Url:** The URL of the talk
- **Views:** Total views for talk

### Data Provided - Transcript:
- **Transcript:** The official English transcript of the talk
- **URL:** The URL of the talk
- test


In [79]:
# Import neccessary libaries

import pandas as pd # Dataframes
import numpy as np # Arrays and Math

# Pandas Profiling Summary report
from pandas_profiling import profile_report

# Visual libraries
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as pe
import matplotlib.pyplot as plt

# set plt inline so plots will automatically run without the .show() command
%matplotlib inline 

# Warnings set to Ignore messages
import warnings
warnings.filterwarnings('ignore')

# Natural Language Processing (NLP)
import string 
import re # RegEX library for specific text isolation

from nltk.tokenize import sent_tokenize, word_tokenize # Tokenize sentences, then words with NLTK
from nltk.corpus import stopwords # Remove stopwords (the, a, is, are, etc.)
from nltk.stem import PorterStemmer # Reduce words back to original base word stem
from nltk import FreqDist # Counts frequency of word distributions

import textstat # Library to count to calculate statistics from text, readability, complexity, etc.

import ast # Abstract Syntax Tree - to help identify and segment information (text) imbedded in lists/dictionaries, etc.

#### Importing Dataset 1: Summary Details

In [2]:
# Import the TED summary dataset as a DataFrame
data1 = pd.read_csv('ted_main.csv')

#### Importing Dataset 2: Transcripts

In [3]:
# Import the TED transcripts dataset as a DataFrame
data2 = pd.read_csv('transcripts.csv')

In [4]:
# Create a copy DataFrame for each dataset, keeping data as backup if needed
TED_summ = data1.copy()

TED_trans = data2.copy()

In [5]:
# View the top 5 rows of data - Summary
TED_summ.head()

,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,url,views
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","[{'id': 865, 'hero': 'https://pe.tedcdn.com/im...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110
1,265,With the same humor and humanity he exuded in ...,977,TED2006,1140825600,43,Al Gore,Al Gore: Averting the climate crisis,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...","[{'id': 243, 'hero': 'https://pe.tedcdn.com/im...",Climate advocate,"['alternative energy', 'cars', 'climate change...",Averting the climate crisis,https://www.ted.com/talks/al_gore_on_averting_...,3200520
2,124,New York Times columnist David Pogue takes aim...,1286,TED2006,1140739200,26,David Pogue,David Pogue: Simplicity sells,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...","[{'id': 1725, 'hero': 'https://pe.tedcdn.com/i...",Technology columnist,"['computers', 'entertainment', 'interface desi...",Simplicity sells,https://www.ted.com/talks/david_pogue_says_sim...,1636292
3,200,"In an emotionally charged talk, MacArthur-winn...",1116,TED2006,1140912000,35,Majora Carter,Majora Carter: Greening the ghetto,1,1151367060,"[{'id': 3, 'name': 'Courageous', 'count': 760}...","[{'id': 1041, 'hero': 'https://pe.tedcdn.com/i...",Activist for environmental justice,"['MacArthur grant', 'activism', 'business', 'c...",Greening the ghetto,https://www.ted.com/talks/majora_carter_s_tale...,1697550
4,593,You've never seen data presented like this. Wi...,1190,TED2006,1140566400,48,Hans Rosling,Hans Rosling: The best stats you've ever seen,1,1151440680,"[{'id': 9, 'name': 'Ingenious', 'count': 3202}...","[{'id': 2056, 'hero': 'https://pe.tedcdn.com/i...",Global health expert; data visionary,"['Africa', 'Asia', 'Google', 'demo', 'economic...",The best stats you've ever seen,https://www.ted.com/talks/hans_rosling_shows_t...,12005869


In [6]:
# View the top 5 rows of data - Transcripts
TED_trans.head()

,transcript,url
0,Good morning. How are you?(Laughter)It's been ...,https://www.ted.com/talks/ken_robinson_says_sc...
1,"Thank you so much, Chris. And it's truly a gre...",https://www.ted.com/talks/al_gore_on_averting_...
2,"(Music: ""The Sound of Silence,"" Simon & Garfun...",https://www.ted.com/talks/david_pogue_says_sim...
3,If you're here today — and I'm very happy that...,https://www.ted.com/talks/majora_carter_s_tale...
4,"About 10 years ago, I took on the task to teac...",https://www.ted.com/talks/hans_rosling_shows_t...


In [7]:
# View the bottom 5 rows of data - Summary
TED_summ.tail()

,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,url,views
2545,17,"Between 2008 and 2016, the United States depor...",476,TED2017,1496707200,4,Duarte Geraldino,Duarte Geraldino: What we're missing in the de...,1,1505851216,"[{'id': 3, 'name': 'Courageous', 'count': 24},...","[{'id': 2596, 'hero': 'https://pe.tedcdn.com/i...",Journalist,"['TED Residency', 'United States', 'community'...",What we're missing in the debate about immigra...,https://www.ted.com/talks/duarte_geraldino_wha...,450430
2546,6,How can you study Mars without a spaceship? He...,290,TED2017,1492992000,3,Armando Azua-Bustos,Armando Azua-Bustos: The most Martian place on...,1,1505919737,"[{'id': 22, 'name': 'Fascinating', 'count': 32...","[{'id': 2491, 'hero': 'https://pe.tedcdn.com/i...",Astrobiologist,"['Mars', 'South America', 'TED Fellows', 'astr...",The most Martian place on Earth,https://www.ted.com/talks/armando_azua_bustos_...,417470
2547,10,Science fiction visions of the future show us ...,651,TED2017,1492992000,1,Radhika Nagpal,Radhika Nagpal: What intelligent machines can ...,1,1506006095,"[{'id': 1, 'name': 'Beautiful', 'count': 14}, ...","[{'id': 2346, 'hero': 'https://pe.tedcdn.com/i...",Robotics engineer,"['AI', 'ants', 'fish', 'future', 'innovation',...",What intelligent machines can learn from a sch...,https://www.ted.com/talks/radhika_nagpal_what_...,375647
2548,32,In an unmissable talk about race and politics ...,1100,TEDxMileHigh,1499472000,1,Theo E.J. Wilson,Theo E.J. Wilson: A black man goes undercover ...,1,1506024042,"[{'id': 11, 'name': 'Longwinded', 'count': 3},...","[{'id': 2512, 'hero': 'https://pe.tedcdn.com/i...",Public intellectual,"['Internet', 'TEDx', 'United States', 'communi...",A black man goes undercover in the alt-right,https://www.ted.com/talks/theo_e_j_wilson_a_bl...,419309
2549,8,With more than half of the world population li...,519,TED2017,1492992000,1,Karoliina Korppoo,Karoliina Korppoo: How a video game might help...,1,1506092422,"[{'id': 21, 'name': 'Unconvincing', 'count': 2...","[{'id': 2682, 'hero': 'https://pe.tedcdn.com/i...",Game designer,"['cities', 'design', 'future', 'infrastructure...",How a video game might help us build better ci...,https://www.ted.com/talks/karoliina_korppoo_ho...,391721


In [8]:
# View the bottom 5 rows of data - Summary
TED_trans.tail()

,transcript,url
2462,"So, Ma was trying to explain something to me a...",https://www.ted.com/talks/duarte_geraldino_wha...
2463,This is a picture of a sunset on Mars taken by...,https://www.ted.com/talks/armando_azua_bustos_...
2464,"In my early days as a graduate student, I went...",https://www.ted.com/talks/radhika_nagpal_what_...
2465,I took a cell phone and accidentally made myse...,https://www.ted.com/talks/theo_e_j_wilson_a_bl...
2466,"We humans are becoming an urban species, so ci...",https://www.ted.com/talks/karoliina_korppoo_ho...


In [9]:
# Inspect the overall shape of the data (rows/columns)
print("DataFrame Shapes:\n")
print("Summary Shape: {}".format(TED_summ.shape))
print("Transcripts Shape: {}".format(TED_trans.shape))

DataFrame Shapes:

Summary Shape: (2550, 17)
Transcripts Shape: (2467, 2)


- The summary file has 2,550 rows of data and 17 columns
- The Transcripts file has slightly less rows (2,467) and only 2 columns

**Both DataFrames have a URL column**, which can likely serve as the same column for merging both datasets into one overall DataFrame.  **83 rows of data will be lost** due to the Transcripts DataFrame having a lower overall row count.

In [10]:
# Use Pandas merge function to combine both datasets on the shared column, URL
TED_combined = pd.merge(TED_summ, TED_trans, how = 'inner', on = 'url') # Use inner to only keep records that match both datasets
print("Combined TED Summary Shape: {}".format(TED_combined.shape))

Combined TED Summary Shape: (2467, 18)


We now have 2,467 combined rows and 18 columns in the combined dataset.

In [11]:
# Examine the combined dataset for missing values and column data types
TED_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2467 entries, 0 to 2466
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   comments            2467 non-null   int64 
 1   description         2467 non-null   object
 2   duration            2467 non-null   int64 
 3   event               2467 non-null   object
 4   film_date           2467 non-null   int64 
 5   languages           2467 non-null   int64 
 6   main_speaker        2467 non-null   object
 7   name                2467 non-null   object
 8   num_speaker         2467 non-null   int64 
 9   published_date      2467 non-null   int64 
 10  ratings             2467 non-null   object
 11  related_talks       2467 non-null   object
 12  speaker_occupation  2461 non-null   object
 13  tags                2467 non-null   object
 14  title               2467 non-null   object
 15  url                 2467 non-null   object
 16  views               2467

There are **no null values** out of the 2,467 entries, however, **Speaker Occupation appears to be missing 6 values** which could be NAs and will require further analysis.

The variables with Object datatypes can be converted to categorical values, saving space and improving EDA presentation.

### Convert Date Colums to DateTime then Object Data Type to Category

In [12]:
# Update Category datatype, for data columns, to DateTime - initial dataset includes seconds so set unit = 's'
TED_combined['film_date'] = pd.to_datetime(TED_combined['film_date'], unit = 's')
TED_combined['published_date'] = pd.to_datetime(TED_combined['film_date'], unit = 's')

In [13]:
# Update Object datatype to Category
for columns in TED_combined:
    if TED_combined[columns].dtype == 'object':
        TED_combined[columns] = TED_combined[columns].astype('category')

In [14]:
# Reexamine DataFrame ensuring all datatypes previously Object are now Categorical
TED_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2467 entries, 0 to 2466
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   comments            2467 non-null   int64         
 1   description         2467 non-null   category      
 2   duration            2467 non-null   int64         
 3   event               2467 non-null   category      
 4   film_date           2467 non-null   datetime64[ns]
 5   languages           2467 non-null   int64         
 6   main_speaker        2467 non-null   category      
 7   name                2467 non-null   category      
 8   num_speaker         2467 non-null   int64         
 9   published_date      2467 non-null   datetime64[ns]
 10  ratings             2467 non-null   category      
 11  related_talks       2467 non-null   category      
 12  speaker_occupation  2461 non-null   category      
 13  tags                2467 non-null   category    

- All columns correctly reflect **Category if previously an Object datatype, excluding Film Date and Published Date** which are set to datetime datatypes.

In [15]:
# Inspect total count of null values in dataset
TED_combined.isnull().sum()

comments              0
description           0
duration              0
event                 0
film_date             0
languages             0
main_speaker          0
name                  0
num_speaker           0
published_date        0
ratings               0
related_talks         0
speaker_occupation    6
tags                  0
title                 0
url                   0
views                 0
transcript            0
dtype: int64

In [16]:
# Inspect total count of NA (NaN) values in dataset
TED_combined.isna().sum()

comments              0
description           0
duration              0
event                 0
film_date             0
languages             0
main_speaker          0
name                  0
num_speaker           0
published_date        0
ratings               0
related_talks         0
speaker_occupation    6
tags                  0
title                 0
url                   0
views                 0
transcript            0
dtype: int64

Due to the fact that there are **only 6 speakers with missing occupations**, all of whom can likely be searched online for generic occupations, these entries can be **manually encoded**.

In [17]:
TED_combined['main_speaker'][TED_combined['speaker_occupation'].isna() == True].values.tolist()

['Sonaar Luthra',
 'Rick Falkvinge',
 'Gary Kovacs',
 'Ryan Holladay',
 'Brian Dettmer',
 'Boniface Mwangi']

In [18]:
# Create a list of speakers with missing occupations
speakers = TED_combined['main_speaker'][TED_combined['speaker_occupation'].isna() == True].values.tolist()

# create an, ordered, input list for respective speaker occupations (from online search)
occupations_input = ['Designer', 'Entrepreneur', 'CEO', 'Inventor', 'Artist', 'Journalist']

# for loop to update each speaker with respective occupation that was missig
for speaker in speakers:
    for occupation in occupations_input:
        TED_combined['speaker_occupation'][TED_combined['main_speaker'] == speaker] = occupation

In [19]:
# Recheck the count of NAs to ensure above loop removed all
TED_combined.isna().sum()

comments              0
description           0
duration              0
event                 0
film_date             0
languages             0
main_speaker          0
name                  0
num_speaker           0
published_date        0
ratings               0
related_talks         0
speaker_occupation    0
tags                  0
title                 0
url                   0
views                 0
transcript            0
dtype: int64

There are no missing values in the Speaker Occupation column.

In [20]:
# Inspect total unique counts by column

TED_combined.nunique()

comments               555
description           2464
duration              1059
event                  320
film_date              690
languages               65
main_speaker          2088
name                  2464
num_speaker              5
published_date         690
ratings               2464
related_talks         2464
speaker_occupation    1421
tags                  2447
title                 2464
url                   2464
views                 2464
transcript            2464
dtype: int64

In [21]:
# Inspect Number of Speakers (per talk) column (only column with low unique values count)
TED_combined['num_speaker'].value_counts()

1    2412
2      46
3       5
4       3
5       1
Name: num_speaker, dtype: int64

**Observations**
- 320 unique events listed (e.g. TED2017)
- There 65 unique languages and 555 unique comments tracked indicate **unique counts of languages/comments per event (e.g. 4 different languages and 10 comments for a given TED talk)**
- As expected, the majority of speakers per talk is 1 (2,412), however there were 46 cases of 2 speakers presenting

In [22]:
# Run descriptive statistics summary for all variables - including categorical
TED_combined.describe(include = 'all').T

,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
comments,2467,NaN,NaN,NaN,NaT,NaT,192.571,284.83,2,63,117,222.5,6404
description,2467,2464,"Shame is an unspoken epidemic, the secret behi...",2,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
duration,2467,NaN,NaN,NaN,NaT,NaT,821.76,334.09,135,582,852,1047,3608
event,2467,320,TED2014,84,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
film_date,2467,690,2017-04-24 00:00:00,64,1984-02-02,2017-08-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN
languages,2467,NaN,NaN,NaN,NaT,NaT,28.2914,8.22976,1,23,28,33,72
main_speaker,2467,2088,Hans Rosling,9,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
name,2467,2464,"Jonathan Haidt: Religion, evolution, and the e...",2,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
num_speaker,2467,NaN,NaN,NaN,NaT,NaT,1.02797,0.208371,1,1,1,1,5
published_date,2467,690,2017-04-24 00:00:00,64,1984-02-02,2017-08-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Initial Sample Observations**
- **Hans Rosling** was the most frequent speaker (9 presentations)
- The most common speaker occupation is Writer (45 occurrences)
- Thbe most frequent date for filming, and subsequently publishing, was **4/24/2017 (64 occurrences)**
- The most frequest event (year) is **TED2014 (84 occurrences)**

### Top 5 Events, Speakers, and Occupations

In [23]:
# Inspect the top 5 speaker occupations
TED_combined['event'].value_counts().head(5)

TED2014    84
TED2009    81
TED2013    77
TED2016    75
TED2015    75
Name: event, dtype: int64

In [24]:
# Inspect the top 5 main speakers
TED_combined['main_speaker'].value_counts().head(5)

Hans Rosling      9
Juan Enriquez     7
Marco Tempest     6
Rives             6
Jonathan Haidt    5
Name: main_speaker, dtype: int64

In [25]:
# Inspect the top 5 speaker occupations
TED_combined['speaker_occupation'].value_counts().head(5)

Writer          45
Journalist      38
Artist          34
Designer        34
Entrepreneur    30
Name: speaker_occupation, dtype: int64

### Top 10 Talks of All Time

In [26]:
# Summary of top 10 viewed TED talks
TED_combined[['main_speaker', 'title', 'views']].sort_values('views', ascending = False).head(10)

,main_speaker,title,views
0,Ken Robinson,Do schools kill creativity?,47227110
1268,Amy Cuddy,Your body language may shape who you are,43155405
649,Simon Sinek,How great leaders inspire action,34309432
800,Brené Brown,The power of vulnerability,31168150
444,Mary Roach,10 things you didn't know about orgasm,22270883
1695,Julian Treasure,How to speak so that people want to listen,21594632
198,Jill Bolte Taylor,My stroke of insight,21190883
5,Tony Robbins,Why we do what we do,20685401
2033,James Veitch,This is what happens when you reply to spam email,20475972
1338,Cameron Russell,"Looks aren't everything. Believe me, I'm a model.",19787465


### Reducing Dataset - Remove Unneccessary Rows/Columns
- Since almost 98% of the Talks involved only 1 speaker, all talks with a higher count can be removed
- The following columns **don't benefit the purposes of this project as it relates to Natural Language Processing:**
    - Related Talks
    - URL
    - Languages
    - Number of Speakers    
    - Views - although helpful in identifying top videos above), offers no NLP analysis value
    - Speaker Name (redundant - already in Main Speaker column)

In [27]:
# Drop the above listed columns from the combined DataFrame
TED_combined.drop(columns = ['related_talks', 'languages', 'url','name','views', 'num_speaker'], axis = 1, inplace = True)

In [28]:
# Inspect updated DataFrame shape
TED_combined.shape

(2467, 12)

After removing the 6 columns mentioned above, the DataFrame now consists of **2.467 rows and 12 columns**.

In [58]:
sent = sent_tokenize(TED_combined['transcript'][0])

In [125]:
# words = []
# for s in range(0,len(sent)):
#     t = word_tokenize(sent[s])
#     words.append(t)

In [36]:
# Create empty count lists and loop through appending textstat counts accordingly
sentence_cnt = []
character_cnt = []
word_cnt = []

for i in TED_combined['transcript']:
    
    sentence_cnt.append(textstat.sentence_count(i))
    character_cnt.append(textstat.char_count(i, ignore_spaces = True))
    word_cnt.append(len(i.split()))

In [37]:
# Add counts to main DataFrame
TED_combined['sentence_cnt'] = sentence_cnt
TED_combined['character_cnt'] = character_cnt
TED_combined['word_cnt'] = word_cnt

In [38]:
TED_combined.head(10)

,comments,description,duration,event,film_date,main_speaker,published_date,ratings,speaker_occupation,tags,title,transcript,sentence_cnt,character_cnt,word_cnt
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,2006-02-25,Ken Robinson,2006-02-25,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,Good morning. How are you?(Laughter)It's been ...,191,14344,3066
1,265,With the same humor and humanity he exuded in ...,977,TED2006,2006-02-25,Al Gore,2006-02-25,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...",Climate advocate,"['alternative energy', 'cars', 'climate change...",Averting the climate crisis,"Thank you so much, Chris. And it's truly a gre...",108,9726,2089
2,124,New York Times columnist David Pogue takes aim...,1286,TED2006,2006-02-24,David Pogue,2006-02-24,"[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...",Technology columnist,"['computers', 'entertainment', 'interface desi...",Simplicity sells,"(Music: ""The Sound of Silence,"" Simon & Garfun...",223,15057,3253
3,200,"In an emotionally charged talk, MacArthur-winn...",1116,TED2006,2006-02-26,Majora Carter,2006-02-26,"[{'id': 3, 'name': 'Courageous', 'count': 760}...",Activist for environmental justice,"['MacArthur grant', 'activism', 'business', 'c...",Greening the ghetto,If you're here today — and I'm very happy that...,152,15235,3015
4,593,You've never seen data presented like this. Wi...,1190,TED2006,2006-02-22,Hans Rosling,2006-02-22,"[{'id': 9, 'name': 'Ingenious', 'count': 3202}...",Global health expert; data visionary,"['Africa', 'Asia', 'Google', 'demo', 'economic...",The best stats you've ever seen,"About 10 years ago, I took on the task to teac...",209,14245,3121
5,672,"Tony Robbins discusses the ""invisible forces"" ...",1305,TED2006,2006-02-02,Tony Robbins,2006-02-02,"[{'id': 7, 'name': 'Funny', 'count': 1102}, {'...",Life coach; expert in leadership psychology,"['business', 'culture', 'entertainment', 'goal...",Why we do what we do,Thank you. I have to tell you I'm both challen...,298,19107,4228
6,919,When two young Mormon missionaries knock on Ju...,992,TED2006,2006-02-24,Julia Sweeney,2006-02-24,"[{'id': 3, 'name': 'Courageous', 'count': 477}...","Actor, comedian, playwright","['Christianity', 'God', 'atheism', 'comedy', '...",Letting go of God,"On September 10, the morning of my seventh bir...",141,12861,2907
7,46,Architect Joshua Prince-Ramus takes the audien...,1198,TED2006,2006-02-23,Joshua Prince-Ramus,2006-02-23,"[{'id': 9, 'name': 'Ingenious', 'count': 195},...",Architect,"['architecture', 'collaboration', 'culture', '...",Behind the design of Seattle's library,I'm going to present three projects in rapid f...,147,15304,3295
8,852,Philosopher Dan Dennett calls for religion -- ...,1485,TED2006,2006-02-02,Dan Dennett,2006-02-02,"[{'id': 3, 'name': 'Courageous', 'count': 562}...","Philosopher, cognitive scientist","['God', 'TED Brain Trust', 'atheism', 'brain',...",Let's teach religion -- all religion -- in sch...,It's wonderful to be back. I love this wonderf...,232,16492,3584
9,900,"Pastor Rick Warren, author of ""The Purpose-Dri...",1262,TED2006,2006-02-25,Rick Warren,2006-02-25,"[{'id': 21, 'name': 'Unconvincing', 'count': 8...","Pastor, author","['Christianity', 'God', 'culture', 'happiness'...",A life of purpose,"I'm often asked, ""What surprised you about the...",231,14545,3441


In [134]:
TED_combined['ratings'][0]

"[{'id': 7, 'name': 'Funny', 'count': 19645}, {'id': 1, 'name': 'Beautiful', 'count': 4573}, {'id': 9, 'name': 'Ingenious', 'count': 6073}, {'id': 3, 'name': 'Courageous', 'count': 3253}, {'id': 11, 'name': 'Longwinded', 'count': 387}, {'id': 2, 'name': 'Confusing', 'count': 242}, {'id': 8, 'name': 'Informative', 'count': 7346}, {'id': 22, 'name': 'Fascinating', 'count': 10581}, {'id': 21, 'name': 'Unconvincing', 'count': 300}, {'id': 24, 'name': 'Persuasive', 'count': 10704}, {'id': 23, 'name': 'Jaw-dropping', 'count': 4439}, {'id': 25, 'name': 'OK', 'count': 1174}, {'id': 26, 'name': 'Obnoxious', 'count': 209}, {'id': 10, 'name': 'Inspiring', 'count': 24924}]"

In [329]:
# USE AST library to retrievbe list inside the string dtype list (ratings)
# TED_combined["ratings"] = TED_combined["ratings"].apply(lambda x: ast.literal_eval(x))

In [330]:
# Examine the list of all Ratings assigned to the first TED talk on the 1st row
TED_combined["ratings"][0]

[{'id': 7, 'name': 'Funny', 'count': 19645},
 {'id': 1, 'name': 'Beautiful', 'count': 4573},
 {'id': 9, 'name': 'Ingenious', 'count': 6073},
 {'id': 3, 'name': 'Courageous', 'count': 3253},
 {'id': 11, 'name': 'Longwinded', 'count': 387},
 {'id': 2, 'name': 'Confusing', 'count': 242},
 {'id': 8, 'name': 'Informative', 'count': 7346},
 {'id': 22, 'name': 'Fascinating', 'count': 10581},
 {'id': 21, 'name': 'Unconvincing', 'count': 300},
 {'id': 24, 'name': 'Persuasive', 'count': 10704},
 {'id': 23, 'name': 'Jaw-dropping', 'count': 4439},
 {'id': 25, 'name': 'OK', 'count': 1174},
 {'id': 26, 'name': 'Obnoxious', 'count': 209},
 {'id': 10, 'name': 'Inspiring', 'count': 24924}]

The Ratings column now reflects a **list of dictionary entries for each rating and respective count assigned** - e.g. Funny at 19,645 for the 1st row.

In [244]:
TED_combined["ratings"]

0       [{'id': 7, 'name': 'Funny', 'count': 19645}, {...
1       [{'id': 7, 'name': 'Funny', 'count': 544}, {'i...
2       [{'id': 7, 'name': 'Funny', 'count': 964}, {'i...
3       [{'id': 3, 'name': 'Courageous', 'count': 760}...
4       [{'id': 9, 'name': 'Ingenious', 'count': 3202}...
                              ...                        
2462    [{'id': 3, 'name': 'Courageous', 'count': 24},...
2463    [{'id': 22, 'name': 'Fascinating', 'count': 32...
2464    [{'id': 1, 'name': 'Beautiful', 'count': 14}, ...
2465    [{'id': 11, 'name': 'Longwinded', 'count': 3},...
2466    [{'id': 21, 'name': 'Unconvincing', 'count': 2...
Name: ratings, Length: 2467, dtype: object

In [278]:
for h in TED_combined["ratings"]:
    print(h[])

{'id': 1, 'name': 'Beautiful', 'count': 4573}
{'id': 3, 'name': 'Courageous', 'count': 139}
{'id': 3, 'name': 'Courageous', 'count': 45}
{'id': 1, 'name': 'Beautiful', 'count': 291}
{'id': 7, 'name': 'Funny', 'count': 1390}
{'id': 3, 'name': 'Courageous', 'count': 721}
{'id': 7, 'name': 'Funny', 'count': 2484}
{'id': 1, 'name': 'Beautiful', 'count': 98}
{'id': 2, 'name': 'Confusing', 'count': 70}
{'id': 1, 'name': 'Beautiful', 'count': 767}
{'id': 9, 'name': 'Ingenious', 'count': 258}
{'id': 3, 'name': 'Courageous', 'count': 96}
{'id': 3, 'name': 'Courageous', 'count': 86}
{'id': 1, 'name': 'Beautiful', 'count': 277}
{'id': 9, 'name': 'Ingenious', 'count': 89}
{'id': 9, 'name': 'Ingenious', 'count': 190}
{'id': 3, 'name': 'Courageous', 'count': 85}
{'id': 3, 'name': 'Courageous', 'count': 150}
{'id': 7, 'name': 'Funny', 'count': 49}
{'id': 2, 'name': 'Confusing', 'count': 13}
{'id': 3, 'name': 'Courageous', 'count': 52}
{'id': 9, 'name': 'Ingenious', 'count': 387}
{'id': 7, 'name': 'Fu

In [308]:
# Loop through each row's ratings counts and add them up, iterating through each dictionary line per row
total_ratings = []

counter = 0
for i in TED_combined["ratings"][counter]:
    print(i['count'])
    total_ratings.append(i['count'])
    counter += 1

19645
4573
6073
3253
387
242
7346
10581
300
10704
4439
1174
209
24924


In [301]:
for i in TED_combined["ratings"][0]:
     print(i.values())

dict_values([7, 'Funny', 19645])
dict_values([1, 'Beautiful', 4573])
dict_values([9, 'Ingenious', 6073])
dict_values([3, 'Courageous', 3253])
dict_values([11, 'Longwinded', 387])
dict_values([2, 'Confusing', 242])
dict_values([8, 'Informative', 7346])
dict_values([22, 'Fascinating', 10581])
dict_values([21, 'Unconvincing', 300])
dict_values([24, 'Persuasive', 10704])
dict_values([23, 'Jaw-dropping', 4439])
dict_values([25, 'OK', 1174])
dict_values([26, 'Obnoxious', 209])
dict_values([10, 'Inspiring', 24924])


In [ ]:
# print out all ratings dictionary values, across all talks/rows
for i in range(0,len(TED_combined)):
    for j in TED_combined["ratings"][i]:
        print(j.values())

In [328]:
# print out all ratings dictionary values, across all talks/rows
total_ratings = []

for i in range(0,len(TED_combined)):
    for j in TED_combined["ratings"][i]:
#         print(j['count'])
        total_ratings.append(j['count'])

total_ratings

[19645,
 4573,
 6073,
 3253,
 387,
 242,
 7346,
 10581,
 300,
 10704,
 4439,
 1174,
 209,
 24924,
 544,
 139,
 62,
 58,
 258,
 113,
 443,
 413,
 132,
 56,
 268,
 116,
 131,
 203,
 964,
 45,
 183,
 60,
 104,
 78,
 395,
 230,
 166,
 27,
 146,
 230,
 54,
 142,
 760,
 291,
 32,
 59,
 105,
 36,
 53,
 380,
 1070,
 132,
 460,
 230,
 35,
 85,
 3202,
 1390,
 942,
 318,
 110,
 72,
 67,
 5433,
 2893,
 4606,
 3736,
 2542,
 248,
 61,
 1102,
 721,
 377,
 301,
 706,
 397,
 285,
 1038,
 5211,
 1350,
 2423,
 441,
 669,
 335,
 477,
 2484,
 320,
 225,
 277,
 72,
 226,
 438,
 316,
 193,
 311,
 308,
 290,
 61,
 195,
 98,
 26,
 34,
 26,
 175,
 121,
 153,
 25,
 37,
 63,
 57,
 9,
 5,
 562,
 70,
 283,
 60,
 177,
 211,
 145,
 623,
 441,
 313,
 764,
 60,
 154,
 83,
 838,
 767,
 476,
 116,
 306,
 111,
 304,
 2556,
 459,
 26,
 1543,
 371,
 245,
 209,
 169,
 258,
 18,
 187,
 608,
 148,
 51,
 34,
 21,
 14,
 160,
 66,
 7,
 39,
 70,
 96,
 16,
 20,
 5,
 201,
 33,
 14,
 41,
 7,
 42,
 20,
 26,
 1,
 188,
 86,
 9,
 16,
 24

In [302]:
# Function to get total number of: ratings, tags, etc. for each talk
def get_total(x):
    count = 0
    for i in x:
        # i = dictionary counts
        count += i['count'] # Increase by each count in ratings per row, summing up the totals
    return count

In [291]:
# Apply the above function to create a total of all Ted Ratings, by row/talk
TED_combined['total_num_rating'] = TED_combined['ratings'].apply(lambda x: get_total(x))

In [292]:
# Sort the dataset by highest count of reviews
TED_combined.sort_values(by = 'total_num_rating', ascending = False)

,comments,description,duration,event,film_date,main_speaker,published_date,ratings,speaker_occupation,tags,title,transcript,sentence_cnt,character_cnt,word_cnt,total_num_rating
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,2006-02-25,Ken Robinson,2006-02-25,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...",Author/educator,"[children, creativity, culture, dance, educati...",Do schools kill creativity?,Good morning. How are you?(Laughter)It's been ...,191,14344,3066,93850
198,2877,Jill Bolte Taylor got a research opportunity f...,1099,TED2008,2008-02-27,Jill Bolte Taylor,2008-02-27,"[{'id': 22, 'name': 'Fascinating', 'count': 14...",Neuroanatomist,"[biology, brain, consciousness, global issues,...",My stroke of insight,I grew up to study the brain because I have a ...,140,11896,2660,70665
1268,2290,"Body language affects how others see us, but i...",1262,TEDGlobal 2012,2012-06-26,Amy Cuddy,2012-06-26,"[{'id': 23, 'name': 'Jaw-dropping', 'count': 3...",Social psychologist,"[body language, brain, business, psychology, s...",Your body language may shape who you are,So I want to start by offering you a free no-t...,227,16409,3602,65968
800,1927,Brené Brown studies human connection -- our ab...,1219,TEDxHouston,2010-06-06,Brené Brown,2010-06-06,"[{'id': 10, 'name': 'Inspiring', 'count': 2144...",Vulnerability researcher,"[TEDx, communication, culture, depression, fea...",The power of vulnerability,"So, I'll start with this: a couple years ago, ...",170,13781,3046,60078
649,1930,Simon Sinek has a simple but powerful model fo...,1084,TEDxPuget Sound,2009-09-17,Simon Sinek,2009-09-17,"[{'id': 21, 'name': 'Unconvincing', 'count': 9...",Leadership expert,"[TEDx, business, entrepreneur, leadership, suc...",How great leaders inspire action,How do you explain when things don't go as we ...,187,13758,2986,56779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2405,26,"Climate change is real, case closed. But there...",787,TED2017,2017-04-24,Kate Marvel,2017-04-24,"[{'id': 24, 'name': 'Persuasive', 'count': 20}...",Climate scientist,"[Anthropocene, biosphere, climate change, envi...",Can clouds buy us more time to solve climate c...,"I am a climate scientist, and I hate weather. ...",108,8203,1754,132
2408,11,The divisiveness plaguing American politics to...,857,TED2017,2017-04-24,Noah Feldman,2017-04-24,"[{'id': 24, 'name': 'Persuasive', 'count': 24}...",Constitutional law scholar,"[United States, government, history, politics]",Hamilton vs. Madison and the birth of American...,If you've been thinking about US politics and ...,66,9348,1859,119
2403,12,Rivers are one of nature's most powerful force...,668,TEDxPSU,2014-03-02,Liz Hajek,2014-03-02,"[{'id': 10, 'name': 'Inspiring', 'count': 11},...",Geoscientist,"[TEDx, ancient world, climate change, environm...",What rivers can tell us about the earth's history,"All right, let's get up our picture of the ear...",94,9339,2066,98
2418,3,"Meet Françoise Mouly, The New Yorker's art dir...",509,TEDNYC,2017-03-08,Françoise Mouly,2017-03-08,"[{'id': 7, 'name': 'Funny', 'count': 10}, {'id...",Art editor,"[TEDNYC, art, creativity, culture, design, humor]",The stories behind The New Yorker's iconic covers,"So 24 years ago, I was brought to The New York...",30,4471,1055,97


In [85]:
str(TED_combined['transcript'][0])

'Good morning. How are you?(Laughter)It\'s been great, hasn\'t it? I\'ve been blown away by the whole thing. In fact, I\'m leaving.(Laughter)There have been three themes running through the conference which are relevant to what I want to talk about. One is the extraordinary evidence of human creativity in all of the presentations that we\'ve had and in all of the people here. Just the variety of it and the range of it. The second is that it\'s put us in a place where we have no idea what\'s going to happen, in terms of the future. No idea how this may play out.I have an interest in education. Actually, what I find is everybody has an interest in education. Don\'t you? I find this very interesting. If you\'re at a dinner party, and you say you work in education — Actually, you\'re not often at dinner parties, frankly.(Laughter)If you work in education, you\'re not asked.(Laughter)And you\'re never asked back, curiously. That\'s strange to me. But if you are, and you say to somebody, you

In [120]:
# Tokenize Transcripts and Remove Stopwords

stop_words = set(stopwords.words('english'))
tokens = word_tokenize(TED_combined['transcript'][0])
result = [i for i in tokens if not i in stop_words]
print(result)

['Good', 'morning', '.', 'How', '?', '(', 'Laughter', ')', 'It', "'s", 'great', ',', "n't", '?', 'I', "'ve", 'blown', 'away', 'whole', 'thing', '.', 'In', 'fact', ',', 'I', "'m", 'leaving', '.', '(', 'Laughter', ')', 'There', 'three', 'themes', 'running', 'conference', 'relevant', 'I', 'want', 'talk', '.', 'One', 'extraordinary', 'evidence', 'human', 'creativity', 'presentations', "'ve", 'people', '.', 'Just', 'variety', 'range', '.', 'The', 'second', "'s", 'put', 'us', 'place', 'idea', "'s", 'going', 'happen', ',', 'terms', 'future', '.', 'No', 'idea', 'may', 'play', 'out.I', 'interest', 'education', '.', 'Actually', ',', 'I', 'find', 'everybody', 'interest', 'education', '.', 'Do', "n't", '?', 'I', 'find', 'interesting', '.', 'If', "'re", 'dinner', 'party', ',', 'say', 'work', 'education', '—', 'Actually', ',', "'re", 'often', 'dinner', 'parties', ',', 'frankly', '.', '(', 'Laughter', ')', 'If', 'work', 'education', ',', "'re", 'asked', '.', '(', 'Laughter', ')', 'And', "'re", 'never

In [121]:
result

['Good',
 'morning',
 '.',
 'How',
 '?',
 '(',
 'Laughter',
 ')',
 'It',
 "'s",
 'great',
 ',',
 "n't",
 '?',
 'I',
 "'ve",
 'blown',
 'away',
 'whole',
 'thing',
 '.',
 'In',
 'fact',
 ',',
 'I',
 "'m",
 'leaving',
 '.',
 '(',
 'Laughter',
 ')',
 'There',
 'three',
 'themes',
 'running',
 'conference',
 'relevant',
 'I',
 'want',
 'talk',
 '.',
 'One',
 'extraordinary',
 'evidence',
 'human',
 'creativity',
 'presentations',
 "'ve",
 'people',
 '.',
 'Just',
 'variety',
 'range',
 '.',
 'The',
 'second',
 "'s",
 'put',
 'us',
 'place',
 'idea',
 "'s",
 'going',
 'happen',
 ',',
 'terms',
 'future',
 '.',
 'No',
 'idea',
 'may',
 'play',
 'out.I',
 'interest',
 'education',
 '.',
 'Actually',
 ',',
 'I',
 'find',
 'everybody',
 'interest',
 'education',
 '.',
 'Do',
 "n't",
 '?',
 'I',
 'find',
 'interesting',
 '.',
 'If',
 "'re",
 'dinner',
 'party',
 ',',
 'say',
 'work',
 'education',
 '—',
 'Actually',
 ',',
 "'re",
 'often',
 'dinner',
 'parties',
 ',',
 'frankly',
 '.',
 '(',
 'L

In [124]:
for line in result:
    re.sub(pattern = '\W', repl = '', string = line)
    print(line)

Good
morning
.
How
?
(
Laughter
)
It
's
great
,
n't
?
I
've
blown
away
whole
thing
.
In
fact
,
I
'm
leaving
.
(
Laughter
)
There
three
themes
running
conference
relevant
I
want
talk
.
One
extraordinary
evidence
human
creativity
presentations
've
people
.
Just
variety
range
.
The
second
's
put
us
place
idea
's
going
happen
,
terms
future
.
No
idea
may
play
out.I
interest
education
.
Actually
,
I
find
everybody
interest
education
.
Do
n't
?
I
find
interesting
.
If
're
dinner
party
,
say
work
education
—
Actually
,
're
often
dinner
parties
,
frankly
.
(
Laughter
)
If
work
education
,
're
asked
.
(
Laughter
)
And
're
never
asked
back
,
curiously
.
That
's
strange
.
But
,
say
somebody
,
know
,
say
,
``
What
?
''
say
work
education
,
see
blood
run
face
.
They
're
like
,
``
Oh
God
,
''
know
,
``
Why
?
``
(
Laughter
)
''
My
one
night
week
.
``
(
Laughter
)
But
ask
education
,
pin
wall
.
Because
's
one
things
goes
deep
people
,
I
right
?
Like
religion
,
money
things
.
So
I
big
interest
educatio

In [114]:
input_str = 'NLTK is a leading platform for building Python programs to work with human language data.'
stop_words = set(stopwords.words('english'))
from nltk.tokenize import word_tokenize
tokens = word_tokenize(input_str)
result = [i for i in tokens if not i in stop_words]
print (result)

['NLTK', 'leading', 'platform', 'building', 'Python', 'programs', 'work', 'human', 'language', 'data', '.']


In [90]:
stemmer= PorterStemmer()
for word in result:
    print(stemmer.stem(word))

nltk
lead
platform
build
python
program
work
human
languag
data
.


In [39]:
TED_combined['transcript'][0].lower().split()

['good',
 'morning.',
 'how',
 'are',
 "you?(laughter)it's",
 'been',
 'great,',
 "hasn't",
 'it?',
 "i've",
 'been',
 'blown',
 'away',
 'by',
 'the',
 'whole',
 'thing.',
 'in',
 'fact,',
 "i'm",
 'leaving.(laughter)there',
 'have',
 'been',
 'three',
 'themes',
 'running',
 'through',
 'the',
 'conference',
 'which',
 'are',
 'relevant',
 'to',
 'what',
 'i',
 'want',
 'to',
 'talk',
 'about.',
 'one',
 'is',
 'the',
 'extraordinary',
 'evidence',
 'of',
 'human',
 'creativity',
 'in',
 'all',
 'of',
 'the',
 'presentations',
 'that',
 "we've",
 'had',
 'and',
 'in',
 'all',
 'of',
 'the',
 'people',
 'here.',
 'just',
 'the',
 'variety',
 'of',
 'it',
 'and',
 'the',
 'range',
 'of',
 'it.',
 'the',
 'second',
 'is',
 'that',
 "it's",
 'put',
 'us',
 'in',
 'a',
 'place',
 'where',
 'we',
 'have',
 'no',
 'idea',
 "what's",
 'going',
 'to',
 'happen,',
 'in',
 'terms',
 'of',
 'the',
 'future.',
 'no',
 'idea',
 'how',
 'this',
 'may',
 'play',
 'out.i',
 'have',
 'an',
 'interest'

In [100]:
TED_combined['tags'][0].lower().split()

["['children',",
 "'creativity',",
 "'culture',",
 "'dance',",
 "'education',",
 "'parenting',",
 "'teaching']"]